# TP2 Cryptographie et Sécurité

TANG Kévin p1501263

Une voyante australienne réputée prétend avoir deviné les 6 numéros gagnants (compris entre 1 et 49) du prochain tirage du loto. Pour des raisons que nous n’expliciterons pas ici, elle souhaite vous en faire profiter (et seulement vous). Etant d’un naturel peu partageur, vous souhaitez sécuriser la communication. Pour ceci, vous décidez d’utiliser le cryptosysteme RSA en transmettant à cette voyante une clé publique (n, e) de taille 2048 (bits).

**1. Sans consigne de votre part, mais quand même familiarisée avec RSA et la classe BigInteger, la voyante vous envoie une encryption de chacun des 6 numeros. Expliquez pourquoi cette façon de procéder n’est absolument pas sécurisée.**

Car si un attaquant parvient à déchiffrer l'encryption pour un seul numéro alors il peut aussi le déchiffrer pour tous les autres numéros.

**2. Afin de pallier la faille précedente, vous suggérez à la voyante de vous envoyer une encryption de la concaténation mc de ces 6 numeros x1, . . . , x6, i.e.**

  **mc = x1∥x2∥x3∥x4∥x5∥x6**

  **Par exemple si x1 = 33, x2 = 12, x3 = 24, x4 = 04, x5 = 08, x6 = 13 alors mc = 331224040813. En considérant qu’un attaquant (écoutant le réseau) dispose de ressources lui permettant d’effectuer une exponentiation modulaire x e mod n en 1ms pour tout x ∈ Zn, estimez le temps nécessaire (en moyenne) à l’attaquant pour retrouver les 6 num ́eros par force brute. Est-ce satisfaisant ?**

Les chiffres sont compris entre 1 et 49.
Il y a donc 49 * 48 * 47 * 46 * 45 * 44 = 10 068 347 520 combinaisons possibles maximum.

En sachant que l'attaquant peut effectuer une exponentiation modulaire en 1ms alors il lui faut 1*49^6 ms pour tester toutes les combinaisons possibles.

10 068 347 520 ms = 10 068 347, 520 s = 116.5318 jours

Cela peut être satisfaisant dans le sens où le tirage sera passé d'ici à ce que l'attaquant décrypte le message.

**3. Même question que la precédente en supposant que la voyante a pris la liberté de classer les numéros par ordre croissant, i.e. x1 < x2 < . . . < x6. Implémenter cette attaque.**

Cela diminue le nombre de possibilités car l'attaquant n'aurait qu'à tester les chiffres supérieurs au précédent.

In [ ]:
from sympy import randprime, mod_inverse
from random import randint
import math
import time

def getprime():
    p = randprime(2 ** (1024 - 1), 2**1024)
    return p

def KeyGen():
   p = getprime()
   q = getprime()
   n= p*q
   phi = (p-1)*(q-1)
   e = 17
   d = inverseModulaire(e,phi)
   pk = (n,e)
   sk = (n,d)
   return pk, sk

def encrypt(m, pk):
  n,e = pk
  c = pow(m, e, n)
  return c

def decrypt(c, pk, sk):
  n,d = sk
  m = pow(c, d, n)
  return m

def inverseModulaire(e, phi_n):
    d = mod_inverse(e, phi_n)
    return d

##################################################################
pk, sk = KeyGen()

mVoyante = 331224040813
keyVoyante = encrypt(mVoyante, pk)

for num1 in range(1,45):
  for num2 in range (num1+1,46):
    for num3 in range (num2+1,47):
      for num4 in range (num3+1,48):
        for num5 in range(num4+1,49):
          for num6 in range(num5+1,50):
            if num1<num2<num3<num4<num5<num6:
              mc= str(num1).zfill(2) + str(num2).zfill(2) + str(num3).zfill(2) + str(num4).zfill(2) + str(num5).zfill(2) + str(num6).zfill(2)
              X = encrypt(int(mc), pk)
              if X == keyVoyante:
                print(X)

**4. Revenons à la question 2. Dans l’euphorie, vous avez malencontreusement choisi e = 17. Pourquoi un tel choix n’est-il pas judicieux ? Proposer une attaque quasi-instantanée permettant à l’attaquant de retrouver les 6 numéros (on utilisera le fait que le message encrypté est trop petit par rapport à n). Implémenter cette attaque.**

Le choix de e doit être premier avec ϕ(n). Si ϕ(n) est pair, alors e doit être impair. Si e est pair, il ne sera jamais premier avec ϕ(n), ce qui peut rendre l'algorithme de chiffrement RSA vulnérable à des attaques.

Le choix de e doit rendre le calcul de l'inverse modulaire d efficace. Si e est trop petit, alors d sera grand, ce qui rendra le déchiffrement lent et inefficace. Par conséquent, il est généralement recommandé de choisir e avec une valeur suffisamment grande pour que le calcul de d soit rapide, mais pas trop grand pour que le chiffrement soit rapide.

**5. Modifier légèrement la méthode d’encryption de RSA permettant de rendre sûre les 4 méthodes précédentes.**